# Tensorflow Estimators

In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
df = pd.read_csv('../data/iris.csv')

In [3]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [4]:
X = df.drop('species', axis='columns')
y = df['species']

column_names = df.columns[:-1]

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [7]:
y_train.head()

60     versicolor
116     virginica
144     virginica
119     virginica
108     virginica
Name: species, dtype: object

In [8]:
labels = ['setosa', 'versicolor', 'virginica']

In [9]:
feature_columns = [
    tf.feature_column.numeric_column(key=key)
    for key in X_train.columns
]

In [10]:
feature_columns

[_NumericColumn(key='sepal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='sepal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='petal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='petal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

## Canned Estimators

In [11]:
rm -rf ./models/iris/

In [12]:
model = tf.estimator.DNNClassifier(
    hidden_units=[10, 10], 
    feature_columns=feature_columns,
    model_dir='./models/iris/DNN/',
    n_classes=3,
    label_vocabulary=labels,
    optimizer='Adam')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models/iris/DNN/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f096fd44470>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [13]:
batch_size=32
train_steps=3000

### Input function

In [14]:
def input_fn(features, labels, batch_size, shuffle=True, repeat=None):
    feature_dict = dict(features)

    if labels is None:
        inputs = feature_dict
    else:
        inputs = (feature_dict, labels)
        
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    
    if shuffle:
        dataset = dataset.shuffle(1000)
    
    dataset = dataset.repeat(repeat)

    return dataset.batch(batch_size)

In [15]:
def train_input_fn():
    return input_fn(X_train, y_train, batch_size)

### Model Training

In [16]:
model.train(
    input_fn=train_input_fn,
    steps=3000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./models/iris/DNN/model.ckpt.
INFO:tensorflow:loss = 61.159294, step = 0
INFO:tensorflow:global_step/sec: 165.945
INFO:tensorflow:loss = 0.17256603, step = 100 (0.606 sec)
INFO:tensorflow:global_step/sec: 198.571
INFO:tensorflow:loss = 2.1592002, step = 200 (0.502 sec)
INFO:tensorflow:global_step/sec: 193.467
INFO:tensorflow:loss = 5.5318046, step = 300 (0.518 sec)
INFO:tensorflow:global_step/sec: 190.367
INFO:tensorflow:loss = 0.16790543, step = 400 (0.524 sec)
INFO:tensorflow:global_step/sec: 202.818
INFO:tensorflow:loss = 1.4711847, step = 500 (0.492 sec)
INFO:tensorflow:global_step/sec: 204.259
INFO:tensorflow:loss = 4.0948234, step = 600 (0.490 sec)
INFO:tensorflow:global_step/sec: 199.752
INFO:tensorfl

### Evaluate model on train and test data

In [17]:
train_eval_result = model.evaluate(
    input_fn=lambda:input_fn(X_train, y_train,
                             batch_size,
                             shuffle=False,
                             repeat=1))

test_eval_result = model.evaluate(
    input_fn=lambda:input_fn(X_test, y_test,
                             batch_size,
                             shuffle=False,
                             repeat=1))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-21-17:12:46
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/iris/DNN/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-21-17:12:46
INFO:tensorflow:Saving dict for global step 3000: accuracy = 0.96190476, average_loss = 0.07840017, global_step = 3000, loss = 2.0580046
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3000: ./models/iris/DNN/model.ckpt-3000
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-21-17:12:47
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/iris/DNN/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018

In [18]:
train_eval_result

{'accuracy': 0.96190476,
 'average_loss': 0.07840017,
 'loss': 2.0580046,
 'global_step': 3000}

In [19]:
test_eval_result

{'accuracy': 0.93333334,
 'average_loss': 0.14648946,
 'loss': 3.2960129,
 'global_step': 3000}

### Predict on new data

In [20]:
predict_x = {
    'sepal_length': [5.1, 5.9, 6.9],
    'sepal_width': [3.3, 3.0, 3.1],
    'petal_length': [1.7, 4.2, 5.4],
    'petal_width': [0.5, 1.5, 2.1],
}

In [21]:
expected = ['setosa', 'versicolor', 'virginica']

In [22]:
predictions = model.predict(
    input_fn=lambda:input_fn(predict_x,
                             labels=None,
                             batch_size=batch_size,
                             shuffle=False,
                             repeat=1))

In [23]:
predictions

<generator object Estimator.predict at 0x7f09532cbfc0>

In [24]:
template = ('\nPrediction is "{}", prob: {:.1f}%, expected "{}"')

for pred_dict, expec in zip(predictions, expected):
    
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    pred_label = labels[class_id]

    print(template.format(pred_label, 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/iris/DNN/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "setosa", prob: 100.0%, expected "setosa"

Prediction is "versicolor", prob: 100.0%, expected "versicolor"

Prediction is "virginica", prob: 95.3%, expected "virginica"


In [25]:
pred_dict

{'logits': array([-10.1251955,   0.6413118,   3.650877 ], dtype=float32),
 'probabilities': array([9.9133899e-07, 4.6995573e-02, 9.5300347e-01], dtype=float32),
 'class_ids': array([2]),
 'classes': array([b'virginica'], dtype=object)}

## Exercise 1

The `tf.estimator` module offers 2 additional canned classifiers:

- `tf.estimator.BaselineClassifier`
- `tf.estimator.LinearClassifier`

What do they do?

Try substituting them in the model definition and re-run the trainin, evaluation and prediction steps.
Is the performance better or worse?

Note: you can set the logging level of tensorflow with `tf.logging.set_verbosity`

In [38]:
model = tf.estimator.BaselineClassifier(
    model_dir='./models/iris/Baseline',
    n_classes=3,
    label_vocabulary=labels
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models/iris/Baseline', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f09da41b208>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [39]:
model.train(
    input_fn=train_input_fn,
    steps=3000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./models/iris/Baseline/model.ckpt.
INFO:tensorflow:loss = 35.155594, step = 0
INFO:tensorflow:global_step/sec: 283.291
INFO:tensorflow:loss = 35.096718, step = 100 (0.355 sec)
INFO:tensorflow:global_step/sec: 293.276
INFO:tensorflow:loss = 35.09646, step = 200 (0.340 sec)
INFO:tensorflow:global_step/sec: 316.788
INFO:tensorflow:loss = 34.967148, step = 300 (0.315 sec)
INFO:tensorflow:global_step/sec: 315.625
INFO:tensorflow:loss = 34.718163, step = 400 (0.317 sec)
INFO:tensorflow:global_step/sec: 313.513
INFO:tensorflow:loss = 34.691345, step = 500 (0.319 sec)
INFO:tensorflow:global_step/sec: 305.082
INFO:tensorflow:loss = 35.18364, step = 600 (0.327 sec)
INFO:tensorflow:global_step/sec: 295.491
INFO:tensorf

In [32]:
train_eval_result = model.evaluate(
    input_fn=lambda:input_fn(X_train, y_train,
                             batch_size,
                             shuffle=False,
                             repeat=1))

test_eval_result = model.evaluate(
    input_fn=lambda:input_fn(X_test, y_test,
                             batch_size,
                             shuffle=False,
                             repeat=1))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-21-17:25:09
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpbtar4p83/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-21-17:25:09
INFO:tensorflow:Saving dict for global step 3000: accuracy = 0.37142858, average_loss = 1.0951169, global_step = 3000, loss = 28.746817
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3000: /tmp/tmpbtar4p83/model.ckpt-3000
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-21-17:25:09
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpbtar4p83/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-

In [33]:
train_eval_result

{'accuracy': 0.37142858,
 'average_loss': 1.0951169,
 'loss': 28.746817,
 'global_step': 3000}

In [34]:
test_eval_result

{'accuracy': 0.24444444,
 'average_loss': 1.1184242,
 'loss': 25.164545,
 'global_step': 3000}

In [46]:
model = tf.estimator.LinearClassifier(
    feature_columns=feature_columns,
    model_dir='./models/iris/Linear',
    n_classes=3,
    label_vocabulary=labels
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models/iris/Linear', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f09da5a6860>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [47]:
model.train(
    input_fn=train_input_fn,
    steps=3000)

train_eval_result = model.evaluate(
    input_fn=lambda:input_fn(X_train, y_train,
                             batch_size,
                             shuffle=False,
                             repeat=1))

test_eval_result = model.evaluate(
    input_fn=lambda:input_fn(X_test, y_test,
                             batch_size,
                             shuffle=False,
                             repeat=1))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./models/iris/Linear/model.ckpt.
INFO:tensorflow:loss = 35.155594, step = 0
INFO:tensorflow:global_step/sec: 228.24
INFO:tensorflow:loss = 10.22942, step = 100 (0.439 sec)
INFO:tensorflow:global_step/sec: 251.025
INFO:tensorflow:loss = 9.105755, step = 200 (0.399 sec)
INFO:tensorflow:global_step/sec: 252.663
INFO:tensorflow:loss = 6.555258, step = 300 (0.395 sec)
INFO:tensorflow:global_step/sec: 255.472
INFO:tensorflow:loss = 7.8035913, step = 400 (0.392 sec)
INFO:tensorflow:global_step/sec: 254.484
INFO:tensorflow:loss = 5.8076077, step = 500 (0.393 sec)
INFO:tensorflow:global_step/sec: 268.707
INFO:tensorflow:loss = 6.6012173, step = 600 (0.372 sec)
INFO:tensorflow:global_step/sec: 258.445
INFO:tensorflow:

In [48]:
train_eval_result

{'accuracy': 0.9809524,
 'average_loss': 0.08569705,
 'loss': 2.2495475,
 'global_step': 3000}

In [49]:
test_eval_result

{'accuracy': 0.9777778,
 'average_loss': 0.108795665,
 'loss': 2.4479024,
 'global_step': 3000}

## Exercise 2

Visualize the training results with tensorboard:

- in a terminal, from the notebook folder, run tensorboard with `tensorboard --logdir=models`
- open the browser at [http://localhost:6006](http://localhost:6006)

